In [1]:
import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import numpy as np
import blackboard
import ka
import ka_rp as karp
import ka_br as kabr
import database_generator as dg
import bb_sfr_opt
import time
import h5py
import train_surrogate_models as tm
import scipy.interpolate

In [3]:
ind, obj = dg.get_data(['height', 'smear', 'pu_content'], ['cycle length', 'pu mass', 'reactivity swing', 'burnup'])

In [6]:
sm = tm.Surrogate_Models()
models = ['lr', 'pr', 'mars', 'ann', 'gpr', 'rf']

for model in models:
    sm.random = 0
    sm.update_database(ind, obj)
    sm.optimize_model(model)
    print(model)
    print('  R2 Score: {}'.format(sm.models[model]['score']))
    print('  MSE Score: {}'.format(sm.models[model]['mse_score']))

lr
  R2 Score: 0.8723439146322037
  MSE Score: 0.09574206402584723
pr
  R2 Score: 0.987404089566852
  MSE Score: 0.009446932824860962
mars
  R2 Score: 0.9296928672599862
  MSE Score: 0.05273466791799721
ann
  R2 Score: 0.9914062116979857
  MSE Score: 0.006453442323288643
gpr
  R2 Score: 0.9823209877538356
  MSE Score: 0.013259259184622993
rf
  R2 Score: 0.9937029493526236
  MSE Score: 0.004722787985532392


In [7]:
print('Predicted: {}'.format(sm.predict(model, [[61.37,51.58,0.7340]])))
print('Predicted: {}'.format(sm.predict(model, [[59.72,50.01,0.8694]])))
print('Predicted: {}'.format(sm.predict(model, [[71.06,55.77,0.3536]])))

Predicted: [[ 100.          755.86844731 1125.60494825   82.71892294]]
Predicted: [[ 100.          817.47804699 1116.06652226   82.54762597]]
Predicted: [[100.         469.13581854 779.84271618  64.95777662]]
